In [5]:
 # Initial imports
import pandas as pd
import hvplot.pandas
from sqlalchemy import create_engine, Column, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
import holoviews as hv
from bokeh.models import HoverTool, NumeralTickFormatter
import quandl
import psycopg2
import streamlit as st

I would like to bypass the database aspect of the previous trials. Select a location and a metric, and then pull the data directly from the API. 
Have to pull available locations and available metrics first, at the very least. Then I would query the data table from the API specifically for those values.

In [12]:
# Retrieve data from Quandl API
quandl.ApiConfig.api_key = 'Hxc-vj2V4iBzuusYo__Q'

In [66]:
regions = quandl.get_table('ZILLOW/REGIONS',region_type = 'state')
regions = regions.sort_values('region')
regions = regions.set_index('region_id')

In [67]:
regions

,region_type,region
region_id,,
4,state,Alabama
3,state,Alaska
8,state,Arizona
6,state,Arkansas
9,state,California
10,state,Colorado
11,state,Connecticut
13,state,Delaware
12,state,District of Columbia


Just doing states for now. Will need to adjust to allow selection of city, county, and/or zip.

In [21]:
region_sel = 'California'

In [38]:
sel_id = regions.loc[regions['region'] == region_sel].index[0]

In [40]:
sel_id

'9'

In [42]:
indicators = quandl.get_table('ZILLOW/INDICATORS')
indicators = indicators.set_index('indicator_id')

In [44]:
indicator_sel = 'ZHVI Single-Family Homes Time Series ($)'

In [46]:
ind_id = indicators.loc[indicators['indicator'] == indicator_sel].index[0]


In [47]:
ind_id

'ZSFH'

In [58]:
data = quandl.get_table('ZILLOW/DATA', indicator_id=ind_id,region_id=sel_id)
data = data.set_index('date')
data = data.sort_values('date')

In [59]:
data

,indicator_id,region_id,value
date,,,
1996-01-31,ZSFH,9,164835.000000
1996-02-29,ZSFH,9,164607.000000
1996-03-31,ZSFH,9,164467.000000
1996-04-30,ZSFH,9,164193.000000
1996-05-31,ZSFH,9,164016.000000
...,...,...,...
2023-04-30,ZSFH,9,739819.812689
2023-05-31,ZSFH,9,743166.386111
2023-06-30,ZSFH,9,748796.698345


In [62]:
data_values = data['value']

In [63]:
data_values

date
1996-01-31    164835.000000
1996-02-29    164607.000000
1996-03-31    164467.000000
1996-04-30    164193.000000
1996-05-31    164016.000000
                  ...      
2023-04-30    739819.812689
2023-05-31    743166.386111
2023-06-30    748796.698345
2023-07-31    756331.350173
2023-08-31    764721.536470
Name: value, Length: 332, dtype: float64

In [78]:
total_period_appreciation = (data_values[-1] - data_values[0]) / data_values[0]
total_period_appreciation

3.6393152938983895

In [91]:
num_years = round(len(data_values.resample('M').sum())/12,1)


In [92]:
num_years

27.7

In [83]:
annualized_appreciation_percentage = (total_period_appreciation**(1 / num_years)) - 1


In [84]:
annualized_appreciation_percentage

0.04779861451029288

In [87]:
t24mo_app = (data_values[-1] - data_values[-24]) / data_values[-24]

In [88]:
t24mo_app

0.08911728854203248

In [89]:
t12mo_app = (data_values[-1] - data_values[-12]) / data_values[-12]

In [90]:
t12mo_app

-0.017548188706267186